**02 • Features Builder (табличка/текст/гео/время/изображения)**

Цель ноутбука — быстро собрать минимальный набор фич без CLI/YAML: всё управляется флагами `FAST`, `SAFE`, `USE_CACHE` прямо в ячейке параметров. Кэширование тяжёлых блоков (TF-IDF, geo-neighbors, эмбеддинги картинок) происходит в `artifacts/features/<block>/<key>/…`, чтобы можно было возобновлять прогон.

Мини-план тура: до обеда успеваем базовые числовые/категориальные, TF-IDF (если текст короткий), гео-гриды; тяжёлые шаги (BallTree-соседи, img-embeddings) — на перерыв или ночь.


In [ ]:
# ——— базовые импорты
import os, sys, json, math, time, gc, warnings
import numpy as np
import pandas as pd
from pathlib import Path

BASE = os.getenv("BASE", ".")
sys.path.append(str(Path(BASE).resolve()))

# ——— прогресс/виджеты (но ноут должен работать и без них)
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kw): return x

try:
    import ipywidgets as W
    _WIDGETS = True
except Exception:
    _WIDGETS = False

# ——— наш фич-слой
from common.cv import make_folds
from common.io import save_set
from common.features import store, assemble
from common.features import (
    num_basic, cat_freq, cat_te_oof, text_tfidf,
    geo_grid, geo_neighbors, time_agg,
    crosses, img_index, img_stats, img_embed
)

from common.cache import make_key
pd.set_option("display.max_colwidth", 120)
warnings.filterwarnings("ignore")

# ——— вспомогалки
def mem_gb(obj=None):
    if obj is None:
        import psutil
        return psutil.Process().memory_info().rss / (1024**3)
    if hasattr(obj, "memory_usage"):
        try:
            return obj.memory_usage(deep=True).sum()/(1024**3)
        except Exception:
            pass
    return np.array(obj).nbytes/(1024**3)

def head(df, n=5):
    display(df.head(n)); print(df.shape, "mem:", round(mem_gb(df), 3), "GB")

print("Widgets:", _WIDGETS, "| Python:", sys.version)



### Панель параметров — что задаём

* Пути к данным и имена ключевых колонок.
* Флаги `FAST` (урезает тяжёлые вещи), `SAFE` (жёсткие анти-утечки), `USE_CACHE`.
* Списки `NUM_COLS`, `CAT_COLS`, `TEXT_COLS`, `MULTI_COLS` (можно оставить `None`, сработает авто-детект).
* Чек-лист блоков, которые включаем/выключаем.
* Ничего лишнего не сохраняется: только кэш фич-блоков. Финальный набор сохраняется отдельно флагом ниже.


In [ ]:
# ——— БАЗОВЫЕ ПАРАМЕТРЫ (редактируй здесь)
DATA_DIR   = "data"            # можно папку
TRAIN_PATH = f"{DATA_DIR}/train.csv"
TEST_PATH  = f"{DATA_DIR}/test.csv"

ID_COL      = "id"
TARGET_COL  = None             # если есть
DATE_COL    = None
LAT_COL     = None
LON_COL     = None
TEXT_COLS   = None             # например ["text"] или None
MULTI_COLS  = None             # например ["tags"] или None

# явные списки числовых/категориальных (можно оставить None, сделаем авто-детект)
NUM_COLS = None
CAT_COLS = None

# глобальные флаги
FAST      = True    # урезает "тяжёлые" параметры
SAFE      = True    # максимально строгие анти-утечки
USE_CACHE = True    # использовать кэш артефактов фич-блоков

# какой сплит (определится ниже): "kfold" | "group" | "time"
SPLIT_KIND = "kfold"
N_SPLITS   = 5
GROUP_COL  = None      # например user_id/item_id для группового
TIME_EMBARGO = None    # напр. "2D" | "3h" если нужно

# какие блоки включить (по умолчанию базовый минимализм)
ACTIVE_BLOCKS = {
    "num_basic": True,
    "cat_freq": True,
    "cat_te_oof": False,    # включай только если понимаешь анти-утечки
    "text_tfidf": False,
    "geo_grid": False,
    "geo_neighbors": False,
    "time_agg": False,
    "crosses": False,
    "img_stats": False,
    "img_embed": False,     # тяжело; запускать на перерыве/ночью
}

# Сохранение итогового набора (опционально)
SAVE_SET   = False
RUN_TAG    = "exp01"

if _WIDGETS:
    # упрощённые виджеты: переключатели блоков/флагов
    toggles = {k: W.Checkbox(value=v, description=k) for k, v in ACTIVE_BLOCKS.items()}
    flags   = {
        "FAST": W.Checkbox(value=FAST, description="FAST"),
        "SAFE": W.Checkbox(value=SAFE, description="SAFE"),
        "USE_CACHE": W.Checkbox(value=USE_CACHE, description="USE_CACHE")
    }
    display(W.HBox(list(flags.values())))
    display(W.GridBox(list(toggles.values()), layout=W.Layout(grid_template_columns="repeat(3, 220px)")))
    def _read_widgets():
        global FAST, SAFE, USE_CACHE, ACTIVE_BLOCKS
        FAST      = flags["FAST"].value
        SAFE      = flags["SAFE"].value
        USE_CACHE = flags["USE_CACHE"].value
        for k in ACTIVE_BLOCKS:
            ACTIVE_BLOCKS[k] = toggles[k].value
    display(W.Button(description="Применить флаги", button_style="info",
                     tooltip="Считать значения чекбоксов")).on_click(lambda _: _read_widgets())


### Загрузка данных, авто-детект колонок, быстрый sanity-чек

* Проверяем наличие колонок, NaN/константы, базовые типы.
* Если `TEXT_COLS=None`, но текст в данных есть — задай список явно.
* Выравниваем порядок столбцов между train/test.


In [ ]:
# ——— чтение
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

print("Train:", train.shape, "| Test:", test.shape)
assert ID_COL in train.columns and ID_COL in test.columns, "ID_COL не найден"
if TARGET_COL:
    assert TARGET_COL in train.columns, "TARGET_COL не найден в train"

# ——— авто-детект типов
def auto_detect_columns(train, exclude):
    num_cols, cat_cols, text_cols = [], [], []
    for c in train.columns:
        if c in exclude:
            continue
        if pd.api.types.is_numeric_dtype(train[c]):
            num_cols.append(c)
        elif pd.api.types.is_string_dtype(train[c]) and train[c].map(lambda x: isinstance(x, str) and len(x)>30).mean()>0.3:
            text_cols.append(c)
        else:
            cat_cols.append(c)
    return num_cols, cat_cols, text_cols

if NUM_COLS is None or CAT_COLS is None or TEXT_COLS is None:
    ex = {ID_COL} | ({TARGET_COL} if TARGET_COL else set())
    n, c, t = auto_detect_columns(train, ex)
    NUM_COLS = n if NUM_COLS is None else NUM_COLS
    CAT_COLS = c if CAT_COLS is None else CAT_COLS
    if TEXT_COLS is None and len(t)>0:
        TEXT_COLS = t

print("NUM_COLS:", NUM_COLS[:10] if NUM_COLS else [])
print("CAT_COLS:", CAT_COLS[:10] if CAT_COLS else [])
print("TEXT_COLS:", TEXT_COLS)

# ——— базовая очистка: одинаковые столбцы, порядок
train = train.copy(); test = test.copy()
train_cols = [c for c in train.columns if c!=TARGET_COL]
test = test[train_cols]  # выравнивание


### Про сплиты и анти-утечки

* Time-сплит: когда есть `DATE_COL` и прогнозируем будущее.
* Group-сплит: если утечки возможны через пользователя/товар.
* KFold по умолчанию, когда нет времени/групп.
* Все обучаемые кодировки (TE/WOE/CTR) считаются строго по OOF.


In [ ]:
FOLDS = make_folds(
    train,
    task="binary",
    n_splits=N_SPLITS,
    split_kind=SPLIT_KIND,
    target_col=TARGET_COL,
    group_col=GROUP_COL,
    date_col=DATE_COL,
    time_embargo=TIME_EMBARGO,
    stratify=True,
    random_state=42
)
print("Folds:", len(FOLDS), "| fold sizes:", [len(v) for _, v in FOLDS][:10])



### Инициализация FeatureStore, соглашение о пакетах

Мы используем класс `FeatureStore` (`common/features/store.py`) как единый реестр вычисленных фичей. Он хранит объекты `FeaturePackage` и умеет прозрачно кэшировать их на диск.

**Структура `FeaturePackage` (`common/features/types.py`):**
* `name` — уникальное имя пакета (строка, используется как ключ в `FeatureStore`).
* `train` / `test` — матрицы или DataFrame c рассчитанными фичами для train/test сплитов.
* `kind` — тип данных: `"dense"`, `"sparse"` или `"mixed"` (подсказывает, как собирать матрицы).
* `cols` — список имён колонок, которые будут добавлены в итоговый датасет.
* `meta` — словарь с сервисной информацией: параметры генерации, время расчёта, флаг OOF, зависимости (`deps`) и т.п.

**Интерфейс `FeatureStore`:**
* `add(pkg)` — регистрирует пакет (ошибается при дубликате имени).
* `get(name)` — возвращает сохранённый пакет по имени.
* `list()` — показывает имена всех зарегистрированных пакетов в памяти.
* `catalog()` — быстрый паспорт пакетов с колонками, типом и метаданными (удобно логировать или смотреть, что уже посчитано).
* `load_cached(name, key)` — пытается прочитать пакет из дискового кэша по ключу (например, хеш параметров). Возвращает `None`, если кэша нет.
* `save_cached(name, key, pkg)` — сохраняет пакет в кэш, чтобы повторно не считать тяжёлые блоки.

**Практика использования в ноутбуке:**
1. Создаём стор: `FS = store.FeatureStore()`.
2. В каждом блоке пытаемся `FS.load_cached(<имя>, cache_key)` и выходим раньше, если кэш найден.
3. Считаем фичи → собираем `FeaturePackage(name=..., train=..., test=..., kind=..., cols=..., meta=...)`.
4. Регистрируем в памяти: `FS.add(pkg)`.
5. По необходимости кладём на диск: `FS.save_cached(<имя>, cache_key, pkg)` при `use_cache=True`.

Такой протокол даёт прозрачное переиспользование фичей между запусками и фиксирует паспорт фичей, который потом удобно собирать через `FS.catalog()`.


In [ ]:
FS = store.FeatureStore()
BUILT = []  # список имён пакетов в порядке построения

def run_block(name, fn, *args, _include=True, _params=None, **kwargs):
    '''
    Универсальная обёртка: печать параметров, тайминг, кэш, добавление в FS.
    '''
    if not _include:
        print(f"— SKIP {name}")
        return
    t0 = time.time()
    print(f"
=== BUILD {name} ===")
    if _params:
        print("params:", json.dumps(_params, ensure_ascii=False))
    pkg = fn(*args, **kwargs)
    FS.add(pkg)
    BUILT.append(pkg.name)
    dt = time.time() - t0
    ntr = pkg.train.shape[1] if hasattr(pkg.train, "shape") else "?"
    print(f"done {name} in {dt:.1f}s | +{ntr} cols | kind={pkg.kind}")


### Числовые фичи (быстро и всегда полезно)

Импутация, лог-масштаб, клиппинг хвостов. Почти всегда включаем. В `FAST=True` оставляем простой набор без скейла/биннинга.


In [ ]:
if ACTIVE_BLOCKS["num_basic"] and NUM_COLS:
    params = dict(prefix="num", num_cols=NUM_COLS, log_cols=None, clip_quant=(0.01,0.99),
                  impute="median", scale=None, use_cache=USE_CACHE)
    run_block("num_basic", num_basic.build, train, test, _include=True, _params=params, **params)
else:
    print("num_basic: пропущен (нет NUM_COLS или выключен)")


### Категориальные частоты (без утечек)

Считают частоты/доли без использования таргета — безопасный базовый сигнал для high-card категорий. Включаем почти всегда.


In [ ]:
if ACTIVE_BLOCKS["cat_freq"] and CAT_COLS:
    params = dict(prefix="catf", cat_cols=CAT_COLS, rare_threshold=0.01, use_cache=USE_CACHE)
    run_block("cat_freq", cat_freq.build, train, test, _params=params, **params)
else:
    print("cat_freq: пропущен (нет CAT_COLS или выключен)")


### OOF-кодировки (target/WOE/CTR) — опасно без OOF!

Включаем только если понимаем анти-утечки и есть таргет. Всегда по `FOLDS`. В `FAST=True` чаще выключаем.


In [ ]:
if ACTIVE_BLOCKS["cat_te_oof"] and CAT_COLS and TARGET_COL:
    params = dict(prefix="te", cat_cols=CAT_COLS, method="target", smoothing="m-estimate", use_cache=USE_CACHE)
    run_block("cat_te_oof", cat_te_oof.build, train, train[TARGET_COL], test, FOLDS, _params=params, **params)
else:
    print("cat_te_oof: пропущен (нет TARGET_COL/CAT_COLS или выключен)")


### Текстовые фичи (TF-IDF)

Если есть текстовые поля: в `FAST=True` поднимаем `min_df`, убираем SVD. Sparse CSR отлично подходит для линейных моделей.


In [ ]:
if ACTIVE_BLOCKS["text_tfidf"] and TEXT_COLS:
    # берём первый текстовый столбец (либо пробеги циклом)
    text_col = TEXT_COLS[0]
    params = dict(text_col=text_col, min_df=5 if FAST else 2, ngram_range=(1,2), use_char=False,
                  svd_k=None if FAST else 256, prefix="tfidf", use_cache=USE_CACHE)
    run_block("text_tfidf", text_tfidf.build, train, test, _params=params, **params)
else:
    print("text_tfidf: пропущен (нет TEXT_COLS или выключен)")


### Гео — гриды и локальная плотность

Гриды по 300/1000 м дают хороший prior; соседи (BallTree) тяжелее. В `FAST=True` можно оставить только один крупный грид.


In [ ]:
if ACTIVE_BLOCKS["geo_grid"] and LAT_COL and LON_COL:
    params = dict(lat_col=LAT_COL, lon_col=LON_COL,
                  steps_m=(1000,) if FAST else (300,1000),
                  prefix="geo", use_cache=USE_CACHE)
    run_block("geo_grid", geo_grid.build, train, test, _params=params, **params)
else:
    print("geo_grid: пропущен (нет LAT/LON или выключен)")


In [ ]:
if ACTIVE_BLOCKS["geo_neighbors"] and LAT_COL and LON_COL:
    params = dict(lat_col=LAT_COL, lon_col=LON_COL, radii_m=(1000,) if FAST else (300,1000),
                  prefix="geonb", use_cache=USE_CACHE)
    run_block("geo_neighbors", geo_neighbors.build, train, test, _params=params, **params)
else:
    print("geo_neighbors: пропущен")


### Время — лаги и роллинги (anti-leak)

Только для временных процессов и прогнозов в будущее. Всегда маска "только прошлое"; при необходимости — эмбарго.


In [ ]:
if ACTIVE_BLOCKS["time_agg"] and DATE_COL:
    params = dict(date_col=DATE_COL, group_cols=[ID_COL], lags=(1,7), rollings=(7,30),
                  folds=FOLDS if SAFE else None, prefix="time", use_cache=USE_CACHE)
    run_block("time_agg", time_agg.build, train, _params=params, **params)
else:
    print("time_agg: пропущен")


### Взаимодействия (кресты)

Ограниченно: используем белый список, чтобы не взорвать размерность.


In [ ]:
if ACTIVE_BLOCKS["crosses"]:
    params = dict(whitelist_num_pairs=None, whitelist_num_cat=None, prefix="x", use_cache=USE_CACHE)
    run_block("crosses", crosses.build, train, test, _params=params, **params)
else:
    print("crosses: пропущен")


### Картинки — быстрые статистики vs эмбеддинги

`img_stats` дешёв и может давать сигнал даже без DL; `img_embed` мощнее, но тяжелее и требует времени/железа. Запускай эмбеддинги на перерыве.


In [ ]:
if ACTIVE_BLOCKS["img_stats"]:
    # нужно построить индекс id->список путей (пример — весь набор train+test)
    all_ids = pd.concat([train[ID_COL], test[ID_COL]]).astype(str).unique()
    try:
        id2 = img_index.build_from_dir(Path(DATA_DIR)/"images", all_ids, pattern="{id}/*.jpg", max_per_id=4)
        params = dict(id_col=ID_COL, id_to_images=id2, prefix="imgstats", use_cache=USE_CACHE)
        run_block("img_stats", img_stats.build, train, test, _params=params, **params)
    except Exception as e:
        print("img_stats: ошибка индекса или чтения:", e)
else:
    print("img_stats: пропущен")


In [ ]:
if ACTIVE_BLOCKS["img_embed"]:
    all_ids = pd.concat([train[ID_COL], test[ID_COL]]).astype(str).unique()
    try:
        id2 = img_index.build_from_dir(Path(DATA_DIR)/"images", all_ids, pattern="{id}/*.jpg", max_per_id=4)
        params = dict(
            id_col=ID_COL, id_to_images=id2,
            backbone="resnet50", image_size=224,
            agg="mean", pool="avg", batch_size=64,
            device="auto", precision="auto", dtype="float16",
            prefix="img", use_cache=USE_CACHE
        )
        run_block("img_embed", img_embed.build, train, test, _params=params, **params)
    except Exception as e:
        print("img_embed: пропущен —", e)
else:
    print("img_embed: пропущен")


### Сборка матриц (Assembler) и паспорт фичей

Собираем `X_dense` для деревьев и `X_sparse` для линейных моделей. Включаем пакеты по `kind`. "Паспорт" покажет, сколько фич добавил каждый блок и текущую память.


In [ ]:
# разложим добавленные пакеты по типу
dense_pkgs  = [name for name in FS.list() if FS.get(name).kind == "dense"]
sparse_pkgs = [name for name in FS.list() if FS.get(name).kind == "sparse"]

print("DENSE packages:", dense_pkgs)
print("SPARSE packages:", sparse_pkgs)

X_dense_tr, X_dense_te, catalog_dense = (None, None, None)
X_sparse_tr, X_sparse_te, catalog_sparse = (None, None, None)

if len(dense_pkgs):
    X_dense_tr, X_dense_te, catalog_dense = assemble.make_dense(FS, include=dense_pkgs)
    print("Dense shapes:", X_dense_tr.shape, X_dense_te.shape, "| mem:", round(mem_gb(X_dense_tr), 3), "GB")

if len(sparse_pkgs):
    X_sparse_tr, X_sparse_te, catalog_sparse = assemble.make_sparse(FS, include=sparse_pkgs)
    print("Sparse shapes:", X_sparse_tr.shape, X_sparse_te.shape)


### Проверки перед сохранением

* Совпадение столбцов train/test, отсутствие NaN/inf (для dense).
* Число строк соответствует train/test.
* Убедись, что размер набора подъёмный для твоей машины.


In [ ]:
if SAVE_SET:
    # аккуратно склей каталоги фич (dense+sparse), если они есть
    feature_catalog = {}
    if catalog_dense:
        feature_catalog = catalog_dense
    if catalog_sparse:
        if feature_catalog:
            feature_catalog = {"dense": catalog_dense, "sparse": catalog_sparse}
        else:
            feature_catalog = catalog_sparse

    save_set(
        run_tag=RUN_TAG,
        Xd_tr=X_dense_tr, Xd_te=X_dense_te,
        Xs_tr=X_sparse_tr, Xs_te=X_sparse_te,
        y=(train[TARGET_COL] if TARGET_COL else None),
        folds=FOLDS,
        feature_catalog=feature_catalog,
        train_ids=train[ID_COL], test_ids=test[ID_COL],
        id_col=ID_COL, target_col=TARGET_COL,
        overwrite=True
    )
else:
    print("Сохранение набора выключено (SAVE_SET=False)")



### Что дальше

* Открой `notebooks/03_model.ipynb` для обучения модели.
* Если сохранял набор — в `03_model` можно читать его с диска; иначе используй переменные из текущего kernel.
* Шпаргалка: мало времени → только `num_basic` + `cat_freq`; быстрый сильный бейзлайн — TF-IDF + линейная модель; тяжёлые `geo_neighbors` и `img_embed` запускай на перерыве.
